This script performs metric calculations for specified features of all participants' gesture data. It then outputs the measurement value calculated for each trial and exports it as a csv file. Two separate files will be exported for freeform and instructional.

The following metrics include:
    - Length (Code blocks 4 & 5)
    - Duration (Code blocks 6 & 7)
    - Speed (Code blocks 8 & 9)
    - Acceleration (Code blocks 10 & 11)
    - Angle (Code blocks 12 & 13)
    - Curvature (Code blocks 14 & 15)
    
Output file details:
    - Two files will be outputted, one for freeform and another for instructional. They will have the same format specified in the following bullet points.
    - Each row contains data for a single trial/stroke.
    - There are two columns for left & right controller.
    - Empty row means that all the calculations are finished for that participant session.

To simplify the file exports, making the following folders for the output metric calculations:
    - GestureAcceleration
    - GestureAngle
    - GestureCurvature
    - GestureDuration
    - GestureLength
    - GestureSpeed

Requirements for calculating any of the metrics in the following order:
    - Libraries must be imported (Code block 1 must run successfully)
    - Dictionary must be created (Code block 2 must run successfully)
    - Mutual functions must be defined (Code block 3 must run successfully)

More details on how the code blocks work can be found in their markdowns.

In [ ]:
import math
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
from scipy.interpolate import BSpline, make_interp_spline
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Overview: Takes in a data directory and stores it in a dictionary that is more efficient to iterate through for creating box plots. Rather than iterating by subject id number in the CleanedData directory, it will iterate by gesture.

Dictionary details:
    - Dictionary will have 13 keys to represent the 13 different gestures. 
    - Each key will have a value of a list that contains two lists. 
    - The two lists will represent the freeform and instruction folder respectively. 
    - Within each of those two lists, they will contain 37 data files paths. 
    - Ex: {'PanLeft': [../session_F_PanSelect_sub1ID.csv, ../session_F_PanSelect_sub2ID.csv, ..][../session_I_PanSelect_sub1ID.csv, ../session_I_PanSelect_sub2ID.csv, ..]}

Cleaned data directory must have the following subfolders/files: ...\CleanedData\Sub#\Session_Sub#_Sess#\\file.csv
The names that are filled in are:
    - '#' in Sub# will be the subject's number
    - 'Session' in Session_Sub#_Sess# is either Freeform or Instructional, '#' in Sub# is the subject's number, and '#' in Sess# is the session number
    - 'file' in file.csv is the name of the file

Requirements:
    - First code block must run successfully
        - Import libraries
    - Cleaned data directory must have the template path: ...\CleanedData\Sub#\Session_Sub#_Sess#\file.csv
        - CleanedData: fixed
        - Sub#: '#' is filled in with the subject's identifier number
        - Session_Sub#_Sess#: first '#' is filled in with the subject's identifier number and second '#' is filled in with the session number for that subject
        - file.csv: 'file' is filled in with the file's name
    - Edit cleaned_data_folder_path to your cleaned data directory path

In [ ]:
''' Edit variable here'''
cleaned_data_folder_path = '..\\CleanedData'

gesture_dict = {}

# Iterates through CleanedData directory to make a dictionary of data where the 13 keys are gesture types and its values 
# are two lists representing freeform and instructional. Each list will contain the path to the 37 csv data files.
for root, sub_folders, files in os.walk(cleaned_data_folder_path):
    for file in files:
        # Splits file name for gesture and session type identification
        file_split = file.split("_")
        file_gesture = file_split[3]
        file_session_type = file_split[2]
         
        # Temporary: ignore box select and subject 1
        if file_gesture == 'BoxSelect':
            continue
        if file_split[5] == str(1):
            continue

        # Ignores the thank you files
        if (file_session_type != 'F') and (file_session_type != 'I'):
            continue

        # If a gesture is not in the dictionary, make the gesture a new key with list values freeform and instructional
        if file_gesture not in gesture_dict:
            freeform_folder = []
            instructional_folder = []
            gesture_dict[file_gesture] = [freeform_folder, instructional_folder]

        if file_session_type == 'F':
            gesture_dict[file_gesture][0].append(os.path.join(root, file))
        else:
            gesture_dict[file_gesture][1].append(os.path.join(root, file))

Overview: Methods that all of the metric calculations will need to use.

This script contains the following methods:
    * get_column_name - Gets the name for a column. Consists of gesture and session type.
    * get_df_for_trial_num - Gets the dataframe for a specific trial number.
    * get_gest_export_path - Gets export path for a single gesture session boxplot data file.
    * read_file - Converts a csv file into a dataframe.

Requirements:
    - First two code blocks must run successfully
        - Import libraries
        - Generate gesture dictionary

In [ ]:
def read_file(file_path):
    """
    Reads a csv file into a dataframe.

    Parameters
    -----
    file_path : str
        path to a csv file

    Returns
    -----
    file : dataframe
        a dataframe of the input csv file
    """
    
    file = pd.read_csv(file_path)
    return file


# metric folder path: gesture_lengths_path = '..\\GesturesLengths'
def get_gest_export_path(metric_folder_path, file_path, gesture, metric_name):
    """
    Gets the export path for a single gesture boxplot data file.

    Parameters
    -----
    metric_folder_path : str
        path to the metric folder to export to
    file_path : str
        path to the original csv data file
    gesture: str
        a gesture key in the gesture_dict variable in code block 2
    metric_name: str
        name of the quantative measurement

    Returns
    -----
    output_path : str
        output_path that the boxplot data file will be exported to
    """

    # Splits the file name based on original file path to identify the session type.
    name_split = os.path.basename(file_path).split("_")
    session = ""
    if name_split[2] == 'F':
        session = "Freeform"
    else:
        session = "Instructional"

    # Create the file name and complete the output path. 
    # Added metric name, session, and gesture type to the file name.
    output_path = metric_folder_path + "\\" + metric_name + '_for'
    for i in range(len(gesture)):
        if (gesture[i].isupper() == True):
            output_path += "_"
        output_path += gesture[i].lower()
    output_path += "_" + session.lower() + '.csv'

    return output_path


def get_column_name(file_path, gesture):
    """
    Gets the name for a column in the boxplot data file, which contains the gesture and session type.

    Parameters
    -----
    file_path : str
        path to the original csv data file
    gesture: str
        a gesture key in the gesture_dict variable in code block 2

    Returns
    -----
    column_name : str
        name of the column based on gesture and session type
    """

    # Splits the file name based on original file path to identify the session type.
    name_split = os.path.basename(file_path).split("_")
    session = ""
    if name_split[2] == 'F':
        session = "Freeform"
    else:
        session = "Instructional"

    # Gets column name based on gesture and session type. 
    # EX) Pan Left Freeform or Zoom In Instructional
    column_name = ""
    for i in range(len(gesture)):
        if ((gesture[i].isupper() == True) and (i != 0)):
            column_name += " "
        column_name += gesture[i]
    column_name += " " + session

    return column_name


def get_df_for_trial_num(df, trial_num):
    """
    Gets a dataframe grouped by the trial number.

    Parameters
    -----
    df : dataframe
        dataframe to analyze
    trial_num : int
        the trial number of the data ('gesture_ui_counter' column in csv file)

    Returns
    -----
    testDF : dataframe
        dataframe that contains rows of the same trial number
    """

    testDF = df.copy()
    # Dropped all rows where the controller was not pressed.
    testDF.drop(testDF[(testDF["trigger_pull_amount_left"] == 0) & (testDF["trigger_pull_amount_right"] == 0)].index, inplace=True)
    # Dropped all rows that did not have the same trial number as the parameter.
    testDF.drop(testDF[(testDF['gesture_counter_UI']) != trial_num].index, inplace=True)
    testDF.reset_index(drop=True, inplace=True)

    return testDF

Overview: Methods to help find the length of a gesture's trial.

This script contains the following methods:
    * calculate_length - Gets the distance between two points in space.
    * get_stroke_length - Gets the total length of a single trial.

Requirements:
    - First two code blocks must run successfully
        - Import libraries
        - Generate gesture dictionary

In [ ]:
def calculate_length(pt1X, pt1Y, pt1Z, pt2X, pt2Y, pt2Z):
    """
    Gets the distance between two points in 3d space.

    Parameters
    -----
    pt1X : float
        x-coordinate of point 1
    pt1Y : float
        y-coordinate of point 1
    pt1Z : float
        z-coordinate of point 1
    pt2X : float
        x-coordinate of point 2
    pt2Y : float
        y-coordinate of point 2
    pt2Z : float
        z-coordinate of point 2

    Returns
    -----
    length : float
        the distance between two points
    """

    xDistance = float(pt2X) - float(pt1X)
    yDistance = float(pt2Y) - float(pt1Y)
    zDistance = float(pt2Z) - float(pt1Z)
    length = math.sqrt((xDistance)**2 + (yDistance)**2 + (zDistance)**2)
    return length


def get_stroke_length(df, trial_num, hand):
    """
    Gets the length of a single trial by adding the total distance between the points that make up the gesture motion.

    Parameters
    -----
    df : dataframe
        file dataframe to analyze
    trial_num : int
        trial number or gesture counter UI to analyze
    hand : char
        left or right controller to analyze the position/translation

    Returns
    -----
    length_sum : float
        the total length of the trial found by adding the distance between every two data points in that trial
    """

    testDF = df.copy()

    # Drops all of the rows where both not pressed & trials that don't have the same trial number.
    if hand == 'l':
        testDF.drop(testDF[(testDF["trigger_pull_amount_left"] == 0)].index, inplace=True)
    else:
        testDF.drop(testDF[(testDF["trigger_pull_amount_right"] == 0)].index, inplace=True)
    testDF.drop(testDF[(testDF['gesture_counter_UI']) != trial_num].index, inplace=True)
    testDF.reset_index(drop=True, inplace=True)

    # Sets the new x, y, and z for pt1 and pt2. 
    # Calculates the distance between those two points and adds that distance to the total sum of the whole trial.
    length_sum = 0
    pt1X = ''
    pt1Y = ''
    pt1Z = ''        
    pt2X = ''
    pt2Y = ''
    pt2Z = ''

    for row in range(len(testDF.loc[:,(hand + "_controller_translation_x")])):
        # Point 1 exists but not point 2
        if row == 0:
            pt1X = float((testDF.loc[:,(hand + "_controller_translation_x")]).iloc[row])
            pt1Y = float((testDF.loc[:,(hand + "_controller_translation_y")]).iloc[row])
            pt1Z = float((testDF.loc[:,(hand + "_controller_translation_z")]).iloc[row])
        else:
            # Point 1 is replaced with Point 2, and the new values for Point 2 is reevaluated.
            if row != 1:
                pt1X = float(pt2X)
                pt1Y = float(pt2Y)
                pt1Z = float(pt2Z)
            pt2X = float((testDF.loc[:,(hand + "_controller_translation_x")]).iloc[row])
            pt2Y = float((testDF.loc[:,(hand + "_controller_translation_y")]).iloc[row])
            pt2Z = float((testDF.loc[:,(hand + "_controller_translation_z")]).iloc[row])
            if ((math.isnan(pt2X)) or (math.isnan(pt2Y)) or (math.isnan(pt2Z))):
                continue
            length_sum += calculate_length(pt1X, pt1Y, pt1Z, pt2X, pt2Y, pt2Z)

    if length_sum == 0:
        length_sum = np.nan
        
    return length_sum

Overview: Main method that outputs gesture length csv data files. Utilizes get_stroke_length() in above method.

Outputs csv files where each file contains data for a single gesture + session type.
    - Will have file name of the format: length_for_gesturetype_sessiontype.csv
        - Example: length_for_zoom_in_freeform.csv

Requirements:
    - The above code block for the length calculation functions must run successfully.
    - The third code block for the mutual functions must run successfully.
    - Edit gesture_length_path variable with your own path to your 'GestureLength' folder

In [ ]:
''' Edit variable here '''
gesture_length_path = '..\\GestureLength'

# Iterate through dictionary and get the length of each gesture trial.
for gesture in gesture_dict:
    for session_type in gesture_dict[gesture]:
        output_path =  get_gest_export_path(gesture_length_path, session_type[0], gesture, 'length')
        column_name = get_column_name(session_type[0], gesture)
        
        # List of data points for right and left hand for a specific gesture and trial
        data_points_left_list = []
        data_points_right_list = []
        
        # Iterates through each file and adds all of its trial data its respective list
        for file_num in range(len(session_type)):
            file_df = pd.read_csv(session_type[file_num])
            max_trial_num = 0
            if len(file_df.index) > 2:
                max_trial_num = int(file_df.iloc[[-2]]['gesture_counter_UI'])
            for trial_num in range(max_trial_num):
                data_points_right_list.append(get_stroke_length(file_df, trial_num+1, 'r'))
                data_points_left_list.append(get_stroke_length(file_df, trial_num+1, 'l'))
            data_points_left_list.append(np.nan)
            data_points_right_list.append(np.nan)

        # Exports single gesture session data file
        single_gesture_session_df = pd.DataFrame()
        single_gesture_session_df[(column_name + " (Left)")] = data_points_left_list
        single_gesture_session_df[(column_name + " (Right)")] = data_points_right_list
        if not os.path.exists(output_path):
            single_gesture_session_df.to_csv(output_path, header=True, index=False)

Overview: Methods to help find the duration of a gesture's trial.

This script contains the following methods:
    * calculate_time_difference - Gets the time difference between the start and the end of the gesture.
    * get_stroke_duration - Gets the total duration of a single trial.

Requirements:
    - First two code blocks must run successfully
        - Import libraries
        - Generate gesture dictionary

In [ ]:
def calculate_time_difference(start_time, end_time):
    """
    Finds the time difference between the start and the end of the gesture.

    Parameters
    -----
    start_time : float
        gesture start time
    end_time : float
        gesture end time

    Returns
    -----
    time_difference : float
        time difference between start and end of gesture
    """

    time_difference = end_time - start_time
    return time_difference


def get_stroke_duration(df, trial_num):
    """
    Finds the total time of a specific trial.

    Parameters
    -----
    df : dataframe
        dataframe to analyze
    trial_num : int
        trial number column in the csv data file

    Returns
    -----
    total_time : float
        how long the gesture lasted
    """

    testDF = df.copy()
    testDF.drop(testDF[(testDF["trigger_pull_amount_left"] == 0) & (testDF["trigger_pull_amount_right"] == 0)].index, inplace=True)
    testDF.drop(testDF[(testDF['gesture_counter_UI']) != trial_num].index, inplace=True)
    testDF.reset_index(drop=True, inplace=True)

    total_duration = 0

    if len(testDF.index) != 0:
        start_time = float(testDF.iloc[[0]]['time'])
        end_time = float(testDF.iloc[[-1]]['time'])
        total_duration = calculate_time_difference(start_time, end_time)

    return total_duration

Overview: Main method that outputs gesture duration csv data files. Utilizes get_stroke_duration() method from above code block.

Outputs csv files where each file contains data for a single gesture + session type.
    - Will have file name of the format: time_for_gesturetype_sessiontype.csv
        - Example: time_for_zoom_in_freeform.csv

Requirements:
    - The above code block for the duration calculation functions must run successfully.
    - The third code block for the mutual functions must run successfully.
    - Edit gesture_time_path variable with your own path to your 'GestureTime' folder

In [ ]:
''' Edit variable here '''
gesture_duration_path = '..\\GestureDuration'

# Iterate through dictionary and get the duration of each gesture trial.
for gesture in gesture_dict:
    for session_type in gesture_dict[gesture]:
        output_path = get_gest_export_path(gesture_duration_path, session_type[0], gesture, 'duration')
        column_name = get_column_name(session_type[0], gesture)

        data_points_list = []
        
        # Iterates through each file and adds all of its trial data its respective list
        for file_num in range(len(session_type)):
            file_df = pd.read_csv(session_type[file_num])
            max_trial_num = 0
            if len(file_df.index) > 2:
                max_trial_num = int(file_df.iloc[[-2]]['gesture_counter_UI'])
            for trial_num in range(max_trial_num):
                data_points_list.append(get_stroke_duration(file_df, trial_num+1))
            data_points_list.append(np.nan)

        # Exports single gesture session data file
        single_gesture_session_df = pd.DataFrame()
        single_gesture_session_df[column_name] = data_points_list
        if not os.path.exists(output_path):
            single_gesture_session_df.to_csv(output_path, header=True, index=False)

Overview: Methods to find the speed of a gesture's trial.

This script contains the following methods:
    * calculate_speed - Gets the average velocity of a trial.

Requirements:
    - First two code blocks must run successfully
        - Import libraries
        - Generate gesture dictionary

In [ ]:
def calculate_speed(length, time):
    """
    Finds the average speed of a stroke.

    Parameters
    -----
    length : float
        length of the stroke
    time : float
        duration of the stroke from start to finish

    Returns
    -----
    speed : float
        length distance over time
    """

    speed = np.nan
    if (time != 0):
        speed = length / time
    return speed

Overview: Main method that outputs gesture speed csv data files. Utilizes calculate_speed() method from above code block.

Outputs csv files where each file contains data for a single gesture + session type.
    - Will have file name of the format: speed_for_gesturetype_sessiontype.csv
        - Example: speed_for_zoom_in_freeform.csv

Requirements:
    - The above code block for the speed calculation functions must run successfully.
    - The third code block for the mutual functions must run successfully.
    - Edit speed_path, length_folder_path, and duration_folder_path variable with your own path to your 'GestureSpeed', 'GestureLength', and 'GestureDuration' folder respectively.
        - You should have this if you ran code blocks 5 & 7 successfully.
        - speed_folder_path is the path where the files will be stored.

In [ ]:
''' Edit 3 variables here '''
speed_folder_path = '..\\GestureSpeed'
length_folder_path = '..\\GestureLength'
duration_folder_path = '..\\GestureDuration'

# List of files in the duration and length folder directory
duration_files_list = os.listdir(duration_folder_path)
length_files_list = os.listdir(length_folder_path)

for file_num in range(len(length_files_list)):
    duration_df = read_file(os.path.join(duration_folder_path, duration_files_list[file_num]))
    length_df = read_file(os.path.join(length_folder_path, length_files_list[file_num]))

    output_path = speed_folder_path + '\\speed_for_'
    single_gesture_session_df = pd.DataFrame()

    # Iterates through each of the column in the length dataframe
    for col_num in range(len(length_df.columns)):
        # Speed dataframe will use the same column name as the length dataframe
        column_name = length_df.columns[col_num]
        data_points_list = []

        # Calculates the speed and adds it to the speed dataframe
        for row_num in range(len(length_df)):
            length = (length_df.iloc[row_num][length_df.columns[col_num]])
            duration = (duration_df.iloc[row_num][duration_df.columns[int(col_num/2)]])
            if (length == np.nan) or (duration == np.nan):
                data_points_list.append(np.nan)
            else:
                data_points_list.append(calculate_speed(length, duration))

        single_gesture_session_df[column_name] = data_points_list

    output_path += length_files_list[file_num][11:]
    if not os.path.exists(output_path):
        single_gesture_session_df.to_csv(output_path, header=True, index=False)

Overview: Methods to find the acceleration of a gesture's trial.

This script contains the following methods:
    * calculate_acceleration - Gets the acceleration of a stroke/trial.

Requirements:
    - First two code blocks must run successfully
        - Import libraries
        - Generate gesture dictionary

In [ ]:
def calculate_acceleration(final_velocity, final_time):
    """
    Finds the acceleration of a stroke.

    Parameters
    -----
    final_velocity : float
        average speed of gesture
    final_time : float
        duration of the stroke from start to finish

    Returns
    -----
    acceleration : float
        rate at which the speed changes over time
    """

    acceleration = np.nan
    if (final_time != 0):
        acceleration = final_velocity / final_time
    return acceleration

Overview: Main method that outputs gesture acceleration csv data files. Utilizes calculate_acceleration() method from above code block.

Outputs csv files where each file contains data for a single gesture + session type.
    - Will have file name of the format: acceleration_for_gesturetype_sessiontype.csv
        - Example: acceleration_for_zoom_in_freeform.csv

Requirements:
    - The above code block for the acceleration calculation methods must run successfully.
    - The third code block for the mutual functions must run successfully.
    - Edit acceleration_path, speed_folder_path, duration_folder_path variable with your own path to your 'GestureAcceleration', 'GestureSpeed', 'GestureDuration' folder respectively
        - You should have the first two if you ran code blocks 7 & 9 successfully.
        - acceleration_folder_path is the path where the files will be stored.

In [ ]:
''' Edit three variables here'''
acceleration_folder_path = '..\\GestureAcceleration'
speed_folder_path = '..\\GestureSpeed'
duration_folder_path = '..\\GestureDuration'

# List of files in the duration and speed folder directory
duration_files_list = os.listdir(duration_folder_path)
speed_files_list = os.listdir(speed_folder_path)

for file_num in range(len(speed_files_list)):
    duration_df = read_file(os.path.join(duration_folder_path, duration_files_list[file_num]))
    speed_df = read_file(os.path.join(speed_folder_path, speed_files_list[file_num]))

    output_path = acceleration_folder_path + '\\acceleration_for_'
    single_gesture_session_df = pd.DataFrame()

    # Iterates through each of the column in the speed dataframe
    for col_num in range(len(speed_df.columns)):
        # Acceleration dataframe will use the same column name as the speed dataframe
        column_name = speed_df.columns[col_num]
        data_points_list = []

        # Calculates the acceleration and adds it to the acceleration dataframe
        for row_num in range(len(speed_df)):
            speed = (speed_df.iloc[row_num][speed_df.columns[col_num]])
            duration = (duration_df.iloc[row_num][duration_df.columns[int(col_num/2)]])
            if (speed == np.nan) and (duration == np.nan):
                data_points_list.append(np.nan)
            else:
                data_points_list.append(calculate_acceleration(speed, duration))

        single_gesture_session_df[column_name] = data_points_list

    output_path += length_files_list[file_num][11:]
    if not os.path.exists(output_path):
        single_gesture_session_df.to_csv(output_path, header=True, index=False)

Overview: Methods to help find the average external angle of a stroke.

The block contains the following methods:
    * calculate_angle - Gets the external angle created by three points in space.
    * get_stroke_angle - Gets average external curvature of a trial/stroke.

Requirements:
    - First two code blocks must run successfully
        - Import libraries
        - Generate gesture dictionary

In [ ]:
def calculate_angle(p1, p2, p3):
    """
    Finds the external angle of three points or 2 vectors.

    Parameters
    -----
    p1 : tuple
        point 1 with x, y, z coordinates
    p2 : tuple
        point 2 with x, y, z coordinates
    p3 : tuple
        point 3 with x, y, z coordinates

    Returns
    -----
    ext_angle : float
        external angle created by the three points
    """
    # Convert points to numpy arrays for easier vector operations
    p1, p2, p3 = np.array(p1), np.array(p2), np.array(p3)

    # Compute the distances between the points
    a_magnitude = np.linalg.norm(p2 - p1)
    b_magnitude = np.linalg.norm(p3 - p2)
    
    ext_angle = np.nan
    if (a_magnitude > 0.0) and (b_magnitude > 0.0):
        # Normalize vector a and b
        normalized_a = (p2-p1)/a_magnitude
        normalized_b = (p3-p2)/b_magnitude
        normalized_dot = np.dot(normalized_a, normalized_b)
        if (normalized_dot) > 1.0:
            normalized_dot = 1.0
        elif (normalized_dot) < -1.0:
            normalized_dot = -1.0
        ext_angle = np.rad2deg(math.acos(normalized_dot))

    return ext_angle


def get_stroke_angle(df, trial_num, hand):
    """
    Gets the angle of a single stroke/trial.
    
    Parameters
    -----
    df : dataframe
        file dataframe to analyze
    trial_num : int
        trial number or gesture counter UI to analyze
    hand : char
        left or right controller to analyze the position/translation

    Returns
    -----
    angle_avg : float
        the sum of all the external angles divided by the number of angles added to that sum
    """

    testDF = df.copy()

    # Drops all of the rows where both not pressed & trials that don't have the same trial number.
    if hand == 'l':
        testDF.drop(testDF[(testDF["trigger_pull_amount_left"] == 0)].index, inplace=True)
    else:
        testDF.drop(testDF[(testDF["trigger_pull_amount_right"] == 0)].index, inplace=True)
    testDF.drop(testDF[(testDF['gesture_counter_UI']) != trial_num].index, inplace=True)
    testDF = testDF[testDF[hand + '_controller_translation_z'].notna()]
    testDF.reset_index(drop=True, inplace=True)

    # Sets the new x, y, and z for pt1 and pt2. 
    # Calculates the external angle created by 3 points.
    angle_sum = 0
    num_angles = 0
    pt1X = ''
    pt1Y = ''
    pt1Z = ''        
    pt2X = ''
    pt2Y = ''
    pt2Z = ''
    pt3X = ''
    pt3Y = ''
    pt3Z = ''

    # Smooths the curve
    x = np.array(testDF[hand + '_controller_translation_x'])
    y = np.array(testDF[hand + '_controller_translation_y'])
    z = np.array(testDF[hand + '_controller_translation_z'])

    # The number of control points and knots
    k = 3  # degree of the B-spline
    t = np.linspace(0, 1, len(testDF))
    # Create the B-spline representation for each dimension
    # Condition checks that make_interp_spline will return valid values
    if (len(t) > 0) and (len(x) > 3):
        spl_x = make_interp_spline(t, x, k=k)
        spl_y = make_interp_spline(t, y, k=k)
        spl_z = make_interp_spline(t, z, k=k)
        # Evaluate the B-spline over a dense set of points for a smooth trajectory
        dense_t = np.linspace(0, 1, 5 * len(testDF))  # points in the smoothed curve; can be adjusted
        x_smooth = spl_x(dense_t)
        y_smooth = spl_y(dense_t)
        z_smooth = spl_z(dense_t)

        # Calculates average angle across 50 points on smoothed curve
        dist_between_pts = int(len(x_smooth) / 50)
        num_pts = 0
        for index in range(len(x_smooth)):
            if dist_between_pts <= 0:
                continue
            if (index % dist_between_pts != 0) :
                continue
            num_pts += 1
            # Sets the x, y, z of points 1, 2, 3
            if num_pts == 1:
                pt1X = float(x_smooth[index])
                pt1Y = float(y_smooth[index])
                pt1Z = float(z_smooth[index])
            elif num_pts == 2:
                pt2X = float(x_smooth[index])
                pt2Y = float(y_smooth[index])
                pt2Z = float(z_smooth[index])
            else:
                if num_pts != 3:
                    pt1X = float(pt2X)
                    pt1Y = float(pt2Y)
                    pt1Z = float(pt2Z)
                    pt2X = float(pt3X)
                    pt2Y = float(pt3Y)
                    pt2Z = float(pt3Z)
                pt3X = x_smooth[index]
                pt3Y = y_smooth[index]
                pt3Z = z_smooth[index]
                if ((math.isnan(pt3X)) or (math.isnan(pt3Y)) or (math.isnan(pt3Z))):
                    break
                p1 = (pt1X, pt1Y, pt1Z)
                p2 = (pt2X, pt2Y, pt2Z)
                p3 = (pt3X, pt3Y, pt3Z)
                angle_to_add = calculate_angle(p1, p2, p3)
                if not math.isnan(angle_to_add):
                    angle_sum += angle_to_add
                    num_angles += 1

    # If a trial is not found, the sum is nan.
    angle_avg = 0
    if angle_sum != 0:
        angle_avg = angle_sum/num_angles
    return angle_avg

Overview: Main method that outputs the average external angle for each gesture stroke/trial. Utilizes get_stroke_angle() method from above code block.

Outputs csv files where each file contains data for a single gesture + session type.
    - Will have file name of the format: angle_for_gesturetype_sessiontype.csv
        - Example: angle_for_zoom_in_freeform.csv

Requirements:
    - The above code block for the acceleration calculation functions must run successfully.
    - The third code block for the mutual functions must run successfully.
    - Edit gesture_angle_path variable below to a path where the files will be stored.

In [ ]:
''' Edit variable here'''
gesture_angle_path = '..\\GestureAngle'

# Iterate through dictionary and get the length of each gesture trial.
for gesture in gesture_dict:
    for session_type in gesture_dict[gesture]:
        output_path =  get_gest_export_path(gesture_angle_path, session_type[0], gesture, 'angle')
        column_name = get_column_name(session_type[0], gesture)
        
        # List of data points for right and left hand for a specific gesture and trial
        data_points_left_list = []
        data_points_right_list = []
        
        # Iterates through each file and adds its 5 trial data its respective list
        for file_num in range(len(session_type)):
            file_df = pd.read_csv(session_type[file_num])
            max_trial_num = 0
            if len(file_df.index) > 2:
                max_trial_num = int(file_df.iloc[[-2]]['gesture_counter_UI'])
            for trial_num in range(max_trial_num):
                data_points_right_list.append(get_stroke_angle(file_df, trial_num+1, 'r'))
                data_points_left_list.append(get_stroke_angle(file_df, trial_num+1, 'l'))
            data_points_left_list.append(np.nan)
            data_points_right_list.append(np.nan)

        # Exports single gesture session data file
        single_gesture_session_df = pd.DataFrame()
        single_gesture_session_df[(column_name + " (Left)")] = data_points_left_list
        single_gesture_session_df[(column_name + " (Right)")] = data_points_right_list
        if not os.path.exists(output_path):
            single_gesture_session_df.to_csv(output_path, header=True, index=False)

Overview: Methods to help find the average curvature of a stroke.

The block contains the following methods:
    * calculate_curvature - Gets the curvature created by three points in space.
    * get_stroke_curvature - Gets average curvature of a trial/stroke.

Requirements:
    - First two code blocks must run successfully
        - Import libraries
        - Generate gesture dictionary

In [ ]:
def calculate_curvature(p1, p2, p3):
    """
    Calculates the curvature at 3 data points.
    
    Parameters
    -----
    p1 : tuple
        point 1 with x, y, z coordinates
    p2 : tuple
        point 2 with x, y, z coordinates
    p3 : tuple
        point 3 with x, y, z coordinates

    Returns
    -----
    curvature : float
        rate of change in direction with respect to distance along the curve
    """

    # Convert points to numpy arrays for easier vector operations
    p1, p2, p3 = np.array(p1), np.array(p2), np.array(p3)

    # Compute the area of the triangle formed by P1, P2, P3
    triangle_area = 0.5 * np.linalg.norm(np.cross(p2 - p1, p3 - p1))

    # Compute the distances between the points
    d12 = np.linalg.norm(p2 - p1)
    d23 = np.linalg.norm(p3 - p2)
    d13 = np.linalg.norm(p3 - p1)

    # Calculate Menger curvature
    curvature = 0.0
    if d12 * d23 * d13 > 0.0:
        curvature = 4 * triangle_area / (d12 * d23 * d13)
    
    return curvature


def get_stroke_curvature(df, trial_num, hand):
    """
    Gets the curvature of a single stroke/trial.
    
    Parameters
    -----
    df : dataframe
        file dataframe to analyze
    trial_num : int
        trial number or gesture counter UI to analyze
    hand : char
        left or right controller to analyze the position/translation

    Returns
    -----
    curvature_avg : float
        the sum of all the external angles divided by the number of angles added to that sum
    """

    testDF = df.copy()

    # Drops all of the rows where both not pressed & trials that don't have the same trial number.
    if hand == 'l':
        testDF.drop(testDF[(testDF["trigger_pull_amount_left"] == 0)].index, inplace=True)
    else:
        testDF.drop(testDF[(testDF["trigger_pull_amount_right"] == 0)].index, inplace=True)
    testDF.drop(testDF[(testDF['gesture_counter_UI']) != trial_num].index, inplace=True)
    testDF = testDF[testDF[hand + '_controller_translation_z'].notna()]
    testDF.reset_index(drop=True, inplace=True)

    # Sets the new x, y, and z for pt1 and pt2. 
    # Calculates the external curvature created by 3 points.
    curvature_sum = 0
    num_curvatures = 0
    pt1X = ''
    pt1Y = ''
    pt1Z = ''        
    pt2X = ''
    pt2Y = ''
    pt2Z = ''
    pt3X = ''
    pt3Y = ''
    pt3Z = ''

    # Smooths the curve
    x = np.array(testDF[hand + '_controller_translation_x'])
    y = np.array(testDF[hand + '_controller_translation_y'])
    z = np.array(testDF[hand + '_controller_translation_z'])

    # The number of control points and knots
    k = 3  # degree of the B-spline
    t = np.linspace(0, 1, len(testDF))
    # Create the B-spline representation for each dimension
    # Condition checks that make_interp_spline will return valid values
    if (len(t) > 0) and (len(x) > 3):
        spl_x = make_interp_spline(t, x, k=k)
        spl_y = make_interp_spline(t, y, k=k)
        spl_z = make_interp_spline(t, z, k=k)
        # Evaluate the B-spline over a dense set of points for a smooth trajectory
        dense_t = np.linspace(0, 1, 5 * len(testDF))  # points in the smoothed curve; can be adjusted
        x_smooth = spl_x(dense_t)
        y_smooth = spl_y(dense_t)
        z_smooth = spl_z(dense_t)

        # Calculates average curvature across 50 points on smoothed curve
        dist_between_pts = int(len(x_smooth) / 50)
        num_pts = 0
        for index in range(len(x_smooth)):
            if dist_between_pts <= 0:
                continue
            if (index % dist_between_pts != 0) :
                continue
            num_pts += 1
            # Sets the x, y, z of points 1, 2, 3
            if num_pts == 1:
                pt1X = float(x_smooth[index])
                pt1Y = float(y_smooth[index])
                pt1Z = float(z_smooth[index])
            elif num_pts == 2:
                pt2X = float(x_smooth[index])
                pt2Y = float(y_smooth[index])
                pt2Z = float(z_smooth[index])
            else:
                if num_pts != 3:
                    pt1X = float(pt2X)
                    pt1Y = float(pt2Y)
                    pt1Z = float(pt2Z)
                    pt2X = float(pt3X)
                    pt2Y = float(pt3Y)
                    pt2Z = float(pt3Z)
                pt3X = x_smooth[index]
                pt3Y = y_smooth[index]
                pt3Z = z_smooth[index]
                if ((math.isnan(pt3X)) or (math.isnan(pt3Y)) or (math.isnan(pt3Z))):
                    break
                
                p1 = (pt1X, pt1Y, pt1Z)
                p2 = (pt2X, pt2Y, pt2Z)
                p3 = (pt3X, pt3Y, pt3Z)
                curvature_to_add = calculate_curvature(p1, p2, p3)
                if not math.isnan(curvature_to_add):
                    curvature_sum += curvature_to_add
                    num_curvatures += 1

    # If a trial is not found, the sum is nan.
    curvature_avg = 0
    if curvature_sum != 0:
        curvature_avg = curvature_sum/num_curvatures
    return curvature_avg

Overview: Main method that outputs the average curvature for each stroke/trial. Utilizes get_stroke_curvature() method from above code block.

Outputs csv files where each file contains data for a single gesture + session type.
    - Will have file name of the format: curvature_for_gesturetype_sessiontype.csv
        - Example: curvature_for_zoom_in_freeform.csv

Requirements:
    - The above code block for the acceleration calculation functions must run successfully.
    - The third code block for the mutual functions must run successfully.
    - Edit gesture_curvature_path variable below to a folderpath where the files will be stored.

In [ ]:
''' Edit variable here '''
gesture_curvature_path = '...\\GestureCurvature'

# Iterate through dictionary and get the length of each gesture trial.
for gesture in gesture_dict:
    for session_type in gesture_dict[gesture]:
        output_path =  get_gest_export_path(gesture_curvature_path, session_type[0], gesture, 'curvature')
        column_name = get_column_name(session_type[0], gesture)
        
        # List of data points for right and left hand for a specific gesture and trial
        data_points_left_list = []
        data_points_right_list = []
        
        # Iterates through each file and adds its 5 trial data its respective list
        for file_num in range(len(session_type)):
            file_df = pd.read_csv(session_type[file_num])
            max_trial_num = 0
            if len(file_df.index) > 2:
                max_trial_num = int(file_df.iloc[[-2]]['gesture_counter_UI'])
            for trial_num in range(max_trial_num):
                data_points_right_list.append(get_stroke_curvature(file_df, trial_num+1, 'r'))
                data_points_left_list.append(get_stroke_curvature(file_df, trial_num+1, 'l'))
            data_points_left_list.append(np.nan)
            data_points_right_list.append(np.nan)

        # Exports single gesture session data file
        single_gesture_session_df = pd.DataFrame()
        single_gesture_session_df[(column_name + " (Left)")] = data_points_left_list
        single_gesture_session_df[(column_name + " (Right)")] = data_points_right_list
        if not os.path.exists(output_path):
            single_gesture_session_df.to_csv(output_path, header=True, index=False)